In [8]:
import numpy as np
import pandas as pd
import pickle
import os
from scipy.integrate import odeint, solve_ivp
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import argparse
from tqdm.notebook import trange, tqdm
from scipy.stats import qmc, norm, truncnorm


In [9]:
def ode_system(t, state, alpha, beta, delta):
    x, y = state
    dotx = alpha * (2 * y + x) - (beta + delta) * x 
    doty = beta * x - (alpha + delta) * y
    return [dotx, doty]

In [10]:
# We assume that the parameters follow a normal distribution with the following mean and sd
# parameters: alpha, beta, delta, x0, y0, eps1, eps2
prior_means = [0.008, 0.01, 0.003, 6.0, 9.0, 0.0, 0.0]
prior_sds = [0.01, 0.01, 0.01, 0.8, 1.5, 0.1, 0.15]
# prior_sds = [0.02, 0.02, 0.01, 0.8, 1.5, 0.1, 0.15]

# prior_means = [0, 0, 0, 6.0, 9.0, 0.0, 0.0]
# prior_sds = [0.01, 0.02, 0.01, 0.8, 1.5, 0.1, 0.15]


In [11]:
df_obs = pd.read_csv('data_1/data.csv')
df_obs.head()

,Comp1,Comp2,Time,Log_Comp1,Log_Comp2,Comp1_noise,Comp2_noise
0,2978.780220,2514.667971,9.030100,7.734868,6.838724,2286.706969,933.297333
1,2850.589991,2653.340649,10.033445,8.210944,8.405288,3681.016640,4470.645982
2,1621.705654,4031.096970,24.080268,7.193893,7.884970,1331.275269,2657.046000
3,992.354387,4835.524659,40.133779,5.977940,9.628795,394.626671,15196.109014
4,924.348734,4936.955813,43.143813,6.517092,8.885697,676.607668,7227.854006


In [12]:
time_space = df_obs['Time']

In [13]:
def sample_truncnorm(mean, sd, lower, upper):
    a, b = (lower - mean) / sd, (upper - mean) / sd
    return truncnorm.rvs(a, b, loc=mean, scale=sd)

In [14]:
suffix = 'train'
plot = True
num_simulations = 50000 if suffix == 'train' else 5000
time_space = df_obs['Time'] # 50
t0 = 0
tmax = 300
time_courses = np.empty((num_simulations, len(time_space), 4)) # 2 is the number of population/features
parameters = np.empty((num_simulations, 7)) # alpha, beta, delta, x0, y0, eps1, eps2

sampler = qmc.LatinHypercube(d=3)
sample = sampler.random(n=num_simulations)

l_bounds = [1e-4, 1e-4, 1e-4]
u_bounds = [0.018, 0.1, 0.015]

#  alpha = 0.0024
#  beta  = 1/20 = 0.05
#  delta = 0.0005

sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

prior_means = [0.008, 0.01, 0.003, 6.0, 9.0, 0.0, 0.0]
prior_sds = [0.01, 0.04, 0.01, 0.8, 1.5, 0.1, 0.15]

for i in trange(num_simulations):
  
  #  alpha = 0.0024
  #  beta  = 1/20 = 0.05
  #  delta = 0.0005
  # Truncated normal allows to enforce non-negativety constraint on the parameters
  # However, further studies are needed to determine whether this is necessary
  alpha = sample_scaled[i, 0]
  beta = sample_scaled[i, 1]
  delta = sample_scaled[i, 2]
  # alpha = sample_truncnorm(prior_means[0], prior_sds[0], 0, np.inf)
  # beta = sample_truncnorm(prior_means[1], prior_sds[1], 0, np.inf)
  # delta = sample_truncnorm(prior_means[2], prior_sds[2], 0, np.inf)
  x0_log = np.random.normal(prior_means[3], prior_sds[3]) 
  y0_log = np.random.normal(prior_means[4], prior_sds[4])
  eps1 = sample_truncnorm(prior_means[5], prior_sds[5], 0, np.inf)
  eps2 = sample_truncnorm(prior_means[6], prior_sds[6], 0, np.inf)

  x0 = np.exp(x0_log)
  y0 = np.exp(y0_log)
  
  assert all(t <= tmax and t >= t0 for t in time_space)
  state0 = [x0, y0]
  res = solve_ivp(ode_system, [t0, tmax], y0=state0, t_eval=time_space, args=(alpha, beta, delta))
  
  # Extract the individual trajectories.
  x, y = res.y

  # Add noise: Multiplicative noise
  log_x_noise = np.log(x) + np.random.normal(0, eps1, len(x))
  log_y_noise = np.log(y) + np.random.normal(0, eps2, len(y))

  x_noise = np.exp(log_x_noise)
  y_noise = np.exp(log_y_noise)

  parameters[i] = [alpha, beta, delta, x0, y0, eps1, eps2]
  time_courses[i] = np.array([x, y, x_noise, y_noise]).T

if not os.path.exists('./data_2'):
  os.makedirs('./data_2')

if plot:
  # Plot the first 30 strains
  fig, axs = plt.subplots(10, 3, figsize=(6, 15))

  for i in range(10):
      for j in range(3):
          ax = axs[i, j]
          y = time_courses[i*3 + j]
          s = np.mean(abs(y), axis = 0)
          ax.plot(time_courses[i*3 + j])  # Change this line

  plt.tight_layout()
  plt.savefig(f'./data_2/two_compt_{suffix}.png')
  plt.show()

# Save parameters
np.save(f'./data_2/two_compt_sims_{suffix}.npy', time_courses)
np.save(f'./data_2/two_compt_params_{suffix}.npy', parameters)

  0%|          | 0/50000 [00:00<?, ?it/s]

KeyboardInterrupt: 